Źródło danych: 


Import Bibliotek

In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Wczytanie danych

In [89]:
dataSet = pd.read_csv("CarBuyers.csv")

Przejrzenie zawartości bazy danych

In [90]:
dataSet.head()

,Manufacturer,Model,Price,Transmission,Power,Engine CC,Fuel,Male,Female,Unknown,Total
0,Ford,Focus,30.619322,5.966102,94.033898,1497.169492,petrol,814172,422731,"56,487",1293390
1,Ford,Fiesta,18.532143,5.714286,68.571429,1166.142857,petrol,554879,631666,"54,057",1240602
2,Volkswagen,Golf,31.242154,6.164835,89.461538,1537.406593,petrol,483216,310604,"47,563",841383
3,Renault,Clio,22.100000,5.615385,75.576923,1219.653846,petrol,241287,312556,"28,004",581847
4,BMW,320i,47.848370,6.444444,126.111111,1995.777778,petrol,408016,115843,"29,125",552984


In [91]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6092 entries, 0 to 6091
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Manufacturer  6092 non-null   object 
 1   Model         6092 non-null   object 
 2   Price         6092 non-null   float64
 3   Transmission  6092 non-null   float64
 4   Power         6092 non-null   float64
 5   Engine CC     6092 non-null   float64
 6   Fuel          6092 non-null   object 
 7   Male          6092 non-null   object 
 8   Female        6092 non-null   object 
 9   Unknown       6092 non-null   object 
 10  Total         6092 non-null   object 
dtypes: float64(4), object(7)
memory usage: 523.7+ KB


In [92]:
dataSet[dataSet["Manufacturer"]=="Ford"]


,Manufacturer,Model,Price,Transmission,Power,Engine CC,Fuel,Male,Female,Unknown,Total
0,Ford,Focus,30.619322,5.966102,94.033898,1497.169492,petrol,814172,422731,"56,487",1293390
1,Ford,Fiesta,18.532143,5.714286,68.571429,1166.142857,petrol,554879,631666,"54,057",1240602
7,Ford,Mondeo,39.973750,1.750000,130.250000,1998.500000,petrol,357452,"69,603","16,550",443605
10,Ford,Ka+,15.460000,5.000000,60.800000,1196.000000,petrol,102708,269428,"20,324",392460
50,Ford,Fusion,19.378909,4.863636,61.272727,1470.909091,petrol,"51,517","41,027","4,395","96,939"
...,...,...,...,...,...,...,...,...,...,...,...
5938,Ford,Mondeo,39.973750,1.750000,130.250000,1998.500000,petrol,"65,305","12,030","2,153","79,488"
5939,Ford,Capri,11.712900,4.000000,69.000000,1864.800000,petrol,"58,188","11,799",804,"70,791"
6042,Ford,Scorpio,30.862909,4.454545,105.545455,2458.909091,petrol,"3,625",408,84,"4,117"
6046,Ford,Maverick,29.151000,5.000000,89.000000,2526.500000,petrol,"2,718",761,161,"3,640"
